In [1]:
#paquetes python
import os
import numpy as np
import copy
import pprint
from math import sqrt
from scipy.linalg import solve_triangular
import inspect
import sys

#paquetes funciones definidas obtenidas de archivos .py
import funciones_factorizacion_QR as fQR

**OBJETIVO:**

El presente notebook tiene por finalidad exhibir el uso y entendimiento de las funciones definidas para resolver un Sistema de Ecuaciones Lineales (SEL) a través de 3 pasos:

    1) Obtención de factorización QR (A=QR)
    2) Implementación del método de eliminación por bloques
    3) Resolución de SEL (Ax=b) con los 2 pasos anteriores
    
Para lograr tal objetivo se presentarán a continuacion las funciones implementadas en python, así como una serie de ejemplos para entender mejor su funcionamiento

## Factorización QR

### Funciones

In [2]:
#Código "crear_matriz_aleatoria"
lines=inspect.getsource(fQR.crear_matriz_aleatoria)
print(lines)

def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstanc

In [3]:
#ayuda "crear_matriz_aleatoria"
help(fQR.crear_matriz_aleatoria)

Help on function crear_matriz_aleatoria in module funciones_factorizacion_QR:

crear_matriz_aleatoria(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango
            indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               np.ndarray que representa matriz (mxn) con numeros al azar en el
                            rango indicado
    
    Ejemplos:
    Crear matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6:
    cr

In [4]:
#código "house"
lines=inspect.getsource(fQR.house)
print(lines)

def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
     

In [5]:
#ayuda "house"
help(fQR.house)

Help on function house in module funciones_factorizacion_QR:

house(x)
    
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de
    householder R, donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz  
    de pxp si v tiene p entradas). El vector v está construido de tal forma que al generar la 
    matriz de reflexión de householder Rf y multiplicarse por x se obtiene una proyección de 
    este último sobre el vector canónico e_1=(1,0,0,...,0) obteniendo un vector con todas sus 
    entradas ingualas  a cero excepto la primera, que de hecho es ||x||

In [6]:
#código "matriz_auxiliar_Arv"
lines=inspect.getsource(fQR.matriz_auxiliar_Arv)
print(lines)

def matriz_auxiliar_Arv(A,ind_singular=False):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    m,n=A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    
    #m contiene el numero de renglones y n el de columna

In [7]:
#Código "matriz_auxiliar_Arv"
help(fQR.matriz_auxiliar_Arv)

Help on function matriz_auxiliar_Arv in module funciones_factorizacion_QR:

matriz_auxiliar_Arv(A, ind_singular=False)
    
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de 
            la matriz R y las entradas de los vectores householder v (excepto la primera), 
            con los cuales se puede calcular la matriz Q. Ambas matrices componentes de la 
            factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener 
                   elemento de factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes 
                   distintas de cero de la matriz R y los vectores householder con los 
                   que se puede obtener la matriz Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogona

In [8]:
#Código "matriz_Q_R"
lines=inspect.getsource(fQR.matriz_Q_R)
print(lines)

def matriz_Q_R(A,ind_singular=False):
    """
    Función que devuelve la matriz R y Q de la factorización QR de una matriz A
    
    params: A    Matriz (mxn)

    return: Q    Matriz Q (mxm) de la factorización A=QR
            R    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    elif A.shape[0]<A.shape[1]:
        sys.exit('El numero de renglones de A tiene que ser igual o mayor al no. de columnas')
    
    #si ind_singular es True, entonces no se devuelve la matriz R ni Q
    #esto se plantea así como un sistema de contról, para evitar comprobar que las matrices
    #sean singulares (y no tengan una única solución) mediante el determinante, que
    #involucra un alto costo computacional para matrices de tamaño representativo
    Arv=matriz_auxiliar_Arv(A,ind_singular)

    if Arv is None:
        return None, None
 

In [9]:
#Ayuda "matriz_Q_R"
help(fQR.matriz_Q_R)

Help on function matriz_Q_R in module funciones_factorizacion_QR:

matriz_Q_R(A, ind_singular=False)
    
    Descripción:
            Función que devuelve la matriz Q y R de la factorización QR de una matriz A
    
    Parametros:
            A     np.ndarray que representa una matriz (mxn)
    
    Resultado:
            R     np.ndarray que representa matriz R (mxm) de la factorización A=QR
            Q     np.ndarray que representa matriz Q (mxm) de la factorización A=QR
    
    
    Para obtener la factorización primero se calcula la matriz auxiliar Arv. La matriz Q 
    (matriz ortogonal tal que Q_transpuesta*Q es igual a la identidad (mxm)
    se calcula como el producto de las matrices Q_j's, las cuales se obtienen a partir
    de las diferentes matrices de reflexión householder utilizadas para generar los ceros
    en las columnas de A para formar la matriz R
    
    Q = Q_1*Q_2*Q_3*Q_4
    
    Nota: Cada matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo 

In [10]:
#Código "Q_j"
lines=inspect.getsource(fQR.Q_j)
print(lines)

def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j

    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m,n=Arv.shape
    Qj=np.eye(m)
    #Para constru

In [11]:
#Ayuda "Q_j"
help(fQR.Q_j)

Help on function Q_j in module funciones_factorizacion_QR:

Q_j(Arv, j)
    
    Descripción:
            Función que calcula la matriz Qj (en el proceso de obtención de 
            factorización QR se van obteniendo n Qj's, que si se multiplican todas 
            da por resultado Q=Q_1*Q_2*...*Q_n)
    
    Parámetros:                            
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                elementos de la matriz R y Q, de la factorización QR
            j       integer que indica el índice de la matriz Q_j
    
    Resultado:
            Qj      np.ndarray que representa la j-esima transformación de householder
    
    La matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo vector 
    de householder como: Q_j = Identidad(mxm) - Beta*v_j*v_j_transpuesta



## Factorización QR

### Ejemplos

In [12]:
#Pra probar el funcionamiento de estas funciones puede definirse una matriz entrada por entrada
#o usar la función crear_matriz_aleatoria

#Ejemplo a mano (asegurandose de que sus entradas sean tipo dobles -aunque sólo se capture
#una parte entera-, pues de lo contrario se tendrán errores por redondeo significativos)
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [13]:
x=np.array([1,2,3.])
print("x=\n",x)

Beta,v=fQR.house(x)
print("\n\nBeta=\n",np.round(Beta,4))
print("\n\nv=\n",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Beta*np.outer(v,v)
print("\n\nR=\n",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\n\nRx=\n",np.round(R@x,4))

x=
 [1. 2. 3.]


Beta=
 0.7327


v=
 [ 1.     -0.7295 -1.0942]


R=
 [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]


Rx=
 [3.7417 0.     0.    ]


In [14]:
#Definase una matriz A
A=fQR.crear_matriz_aleatoria(6,4,6,-6)
print('\n\nA=\n',np.round(A,4))

#Calculamos la matriz auxiliar Arv para visualizarla
Arv=fQR.matriz_auxiliar_Arv(A)
print('\n\nArv=\n',np.round(Arv,4))

#Obtenemos la matriz Q y R a partir de Arv
Q,R=fQR.matriz_Q_R(A)
print('\n\nQ=\n',np.round(Q,4))
print('\n\nR=\n',np.round(R,4))

#Checamos que QR=A
print('\n\nQ@R=\n',np.round(Q@R,4))
print('\n\nA=\n',np.round(A,4))



A=
 [[ 1.0044  3.7357 -0.4083 -0.9154]
 [ 3.5217 -3.4771 -5.9838  4.4937]
 [-0.5128  4.5858  3.8352 -0.8523]
 [-2.7381  0.1452  1.8612  5.4076]
 [ 2.1032  5.0831 -0.5211  3.9373]
 [-2.9406 -3.055  -5.2158  3.4916]]


Arv=
 [[ 5.8517  1.4409 -2.4445 -0.2478]
 [-0.7265  8.9549  6.0242 -1.3919]
 [ 0.1058 -0.4034  6.2733 -5.1334]
 [ 0.5649  0.1069 -0.2661  7.0952]
 [-0.4339 -0.5647  0.809  -9.436 ]
 [ 0.6067  0.4131  0.3045  2.156 ]]


Q=
 [[ 0.1717  0.3895 -0.3723 -0.3159 -0.5385 -0.5389]
 [ 0.6018 -0.4851 -0.2535  0.3758  0.1947 -0.3988]
 [-0.0876  0.5262  0.0719  0.0321  0.7205 -0.436 ]
 [-0.4679  0.0915  0.0265  0.7829 -0.3117 -0.2487]
 [ 0.3594  0.5098 -0.4326  0.3545  0.0288  0.5453]
 [-0.5025 -0.2603 -0.7773 -0.1389  0.2344  0.0359]]


R=
 [[ 5.8517  1.4409 -2.4445 -0.2478]
 [ 0.      8.9549  6.0242 -1.3919]
 [ 0.      0.      6.2733 -5.1334]
 [-0.     -0.     -0.      7.0952]
 [-0.      0.     -0.     -0.    ]
 [-0.      0.      0.      0.    ]]


Q@R=
 [[ 1.0044  3.7357 -0.4083 

In [15]:
#Comparemos con el código de numpy
Q_np, R_np=np.linalg.qr(A)
#notamos que numpy obtiene con signos contrarios Q y R
print('Q_np=\n',np.round(Q_np,4))
#Además R no guarda toda la matriz R de mxn, sino sólo la parte superior de nxn
print('\nR_np=\n',np.round(R_np,4))
#la multiplicación da por resultado A
print('\nQ_np@R_np=\n',np.round(Q_np@R_np,4))

Q_np=
 [[-0.1717  0.3895  0.3723  0.3159]
 [-0.6018 -0.4851  0.2535 -0.3758]
 [ 0.0876  0.5262 -0.0719 -0.0321]
 [ 0.4679  0.0915 -0.0265 -0.7829]
 [-0.3594  0.5098  0.4326 -0.3545]
 [ 0.5025 -0.2603  0.7773  0.1389]]

R_np=
 [[-5.8517 -1.4409  2.4445  0.2478]
 [ 0.      8.9549  6.0242 -1.3919]
 [ 0.      0.     -6.2733  5.1334]
 [ 0.      0.      0.     -7.0952]]

Q_np@R_np=
 [[ 1.0044  3.7357 -0.4083 -0.9154]
 [ 3.5217 -3.4771 -5.9838  4.4937]
 [-0.5128  4.5858  3.8352 -0.8523]
 [-2.7381  0.1452  1.8612  5.4076]
 [ 2.1032  5.0831 -0.5211  3.9373]
 [-2.9406 -3.055  -5.2158  3.4916]]


In [16]:
#Observese que Q=(Q_4*Q_3*Q_2*Q_1)_transpuesta
np.round(np.transpose(fQR.Q_j(Arv,4)@fQR.Q_j(Arv,3)@fQR.Q_j(Arv,2)@fQR.Q_j(Arv,1)),4)

array([[ 0.1717,  0.3895, -0.3723, -0.3159, -0.5385, -0.5389],
       [ 0.6018, -0.4851, -0.2535,  0.3758,  0.1947, -0.3988],
       [-0.0876,  0.5262,  0.0719,  0.0321,  0.7205, -0.436 ],
       [-0.4679,  0.0915,  0.0265,  0.7829, -0.3117, -0.2487],
       [ 0.3594,  0.5098, -0.4326,  0.3545,  0.0288,  0.5453],
       [-0.5025, -0.2603, -0.7773, -0.1389,  0.2344,  0.0359]])

In [17]:
#Puede comprobarse que Q_transpuesta*Q es igual a la identidad
print(np.round(np.transpose(Q)@Q,4))

[[ 1. -0. -0.  0.  0. -0.]
 [-0.  1.  0. -0.  0.  0.]
 [-0.  0.  1. -0. -0.  0.]
 [ 0. -0. -0.  1.  0. -0.]
 [ 0.  0. -0.  0.  1.  0.]
 [-0.  0.  0. -0.  0.  1.]]


In [18]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
A=fQR.crear_matriz_aleatoria(5,5,5,-5,True)
print('\n\nA=\n',np.round(A,4))

m=A.shape[0]
n=A.shape[1]
Q=np.eye(m)
R=copy.copy(A)
Arv=fQR.matriz_auxiliar_Arv(A)
for j in range(n):
    if j==0:
        cadena='Q'+str(j+1)
    else:
        cadena='Q'+str(j+1) + ' @ ' + cadena
    Qj=fQR.Q_j(Arv,j+1)
    print('\nQ',j+1,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=fQR.Q_j(Arv,j+1)@R
    print('\n',cadena,'@ A=\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))
print('\nQ@R:\n',np.round(Q@R,4))
print('\nA:\n',np.round(A,4))



A=
 [[ 4. -1. -3. -3. -1.]
 [-5. -1. -4. -2.  4.]
 [ 5.  0. -1.  2. -1.]
 [ 2.  2.  0.  3.  4.]
 [ 0. -3. -1. -3.  3.]]

Q 1 :
 [[ 0.4781 -0.5976  0.5976  0.239   0.    ]
 [-0.5976  0.3157  0.6843  0.2737  0.    ]
 [ 0.5976  0.6843  0.3157 -0.2737  0.    ]
 [ 0.239   0.2737 -0.2737  0.8905  0.    ]
 [ 0.      0.      0.      0.      1.    ]]

 Q1 @ A=
 [[ 8.3666  0.5976  0.3586  1.6733 -2.51  ]
 [-0.      0.8294 -0.1543  3.3512  2.271 ]
 [ 0.     -1.8294 -4.8457 -3.3512  0.729 ]
 [ 0.      1.2683 -1.5383  0.8595  4.6916]
 [ 0.     -3.     -1.     -3.      3.    ]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      0.2167 -0.4781  0.3314 -0.784 ]
 [ 0.     -0.4781  0.7082  0.2023 -0.4785]
 [ 0.      0.3314  0.2023  0.8598  0.3317]
 [ 0.     -0.784  -0.4785  0.3317  0.2153]]

 Q2 @ Q1 @ A=
 [[ 8.3666  0.5976  0.3586  1.6733 -2.51  ]
 [-0.      3.8266  2.5573  4.9652 -0.6533]
 [ 0.      0.     -3.1908 -2.3661 -1.0558]
 [-0.      0.     -2.6857  0.1766  5.929 ]
 [ 0.      0.  

In [19]:
#Se prueba un ejemplo más grande la función matriz_auxiliar_Arv
#que es base para la obtención de Q y R
A=fQR.crear_matriz_aleatoria(11**2,10**2,10,-10,False)
Arv=fQR.matriz_auxiliar_Arv(A)
Arv

array([[ 5.96317862e+01,  2.22802094e+00, -3.49768147e+00, ...,
        -3.06589752e+00, -5.76594374e+00, -1.08240200e+00],
       [ 1.08175499e-01,  6.47415544e+01, -7.55862460e+00, ...,
        -5.34235642e+00,  1.37419790e+01,  4.83551946e+00],
       [ 6.65085248e-02, -1.50151566e-01,  6.64749139e+01, ...,
        -6.28138861e+00,  5.68003491e+00,  1.24241645e+00],
       ...,
       [-7.22637229e-03,  4.89241486e-02, -1.33029927e-01, ...,
        -2.04904897e-01,  2.88443988e-02,  2.48068657e-01],
       [ 8.32565790e-02, -7.08812735e-02,  1.43735082e-01, ...,
        -5.31205595e-02, -2.68746555e-01, -3.80829295e-03],
       [-1.19905557e-01,  4.56747270e-02, -8.54579003e-02, ...,
         8.55698475e-02, -2.64537656e-01,  4.60791843e-03]])

In [20]:
#Se puede verificar si una matriz es singular mediante el determinante (si
#este es igual a cero). La propiedad de singularidad o no singularidad se
#preserva bajo las operaciones efectuadas a la matriz A para obtener R (QA=R)
#y podemos checar si esta matriz es singular si alguno de los elementos
#de su diagonal es igual a cero
#La función fQR.matriz_auxiliar_Arv, fQR.matriz_Q_R tienen un argumentos adiciona
#para que en caso de que se encuentre una matriz singular suspenda los cálculos

#Obtención de matriz Q y R de una matriz singular A
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype='d')
print('A\n',A)
Q,R=fQR.matriz_Q_R(A)
print('\nQ:\n',np.round(Q,2))
#Nótese como existe un cero en la diagonal de esta matriz
print('\nR:\n',np.round(R,2))
print('\nQ@R:\n',Q@R)

#Al momento de calcular la factorización QR podemos solicitar que nos indique
#si la matriz es singular y suspenda los cálculos (regresando valores None)
fQR.matriz_Q_R(A,1)

A
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

Q:
 [[ 0.12  0.9   0.41]
 [ 0.49  0.3  -0.82]
 [ 0.86 -0.3   0.41]]

R:
 [[ 8.12  9.6  11.08]
 [-0.    0.9   1.81]
 [-0.    0.    0.  ]]

Q@R:
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


(None, None)

## Solución de Sistemas de Ecuaciones Lineales

### Funciones

In [21]:
#Código Solucion_SEL_QR_nxn
lines=inspect.getsource(fQR.Solucion_SEL_QR_nxn)
print(lines)

def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones 

In [22]:
#Ayuda Solucion_SEL_QR_nxn
help(fQR.Solucion_SEL_QR_nxn)

Help on function Solucion_SEL_QR_nxn in module funciones_factorizacion_QR:

Solucion_SEL_QR_nxn(A, b)
    
    Descripción:
            Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con 
        n ecuaciones y n incognitas
    
    Parámetros:                            
            A      Matriz (nxn) que representa los coeficientas de las ecuaciones 
               elementos de la matriz R y Q, de la factorización QR
            b      vector (nx1) constantes del sistema
    
    Resultado:
            x      vector que satisface (Ax=b)
    
    El Sistema de Ecuaciones Lineales establecido como Ax=b, es resuelto con esta función
    mediante la factorización QR como a continuación se explica:
    
      Si A*x = b y A = Q*R, con A matriz (mxn), b (mx1) y Q una matriz ortogonal (mxm) y 
                                R (mxn) una matriz triangular superior
      entonces Q*R*x = b.
      
      Aprovechando la propiedad de Q^(-1)=Q, es decir, Q^(-1)*Q=Ide

## Eliminación por bloques

### Funciones

In [23]:
#Código "crear_bloques"
lines=inspect.getsource(fQR.crear_bloques)
print(lines)

def crear_bloques(A, b, m1=False, n1=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            m1  Numero de columnas que tendrá el 1er bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A deb

In [24]:
#Ayuda "crear_bloques"
help(fQR.crear_bloques)

Help on function crear_bloques in module funciones_factorizacion_QR:

crear_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que divide la matriz A(mxn) y el vector b(mx1) en cuatro submatrices y dos vectores respectivamente
    
    Parámetros:                            
      A     Matriz (mxn) 
      b     vector (mx1)
      m1    Número de renglones del los bloques superiores
      n1    Número de columnas de los bloques de la izquierda
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
      b es un parámetro opcional, si no es introducido solo regresa las submatrices de A
    
    
    Resultado:
      A11   numpy.ndarray que representa la matriz de dimensión m1xn1
      A12   numpy.ndarray que representa la matriz de dimensión m2xn1
      A12   numpy.ndarray que representa la matriz de dimensión m1xn2
      A22   numpy.ndarray que representa la matriz 

In [25]:
#Código "eliminacion_bloques"
lines=inspect.getsource(fQR.eliminacion_bloques)
print(lines)

def eliminacion_bloques(A,b, m1=False, n1=False):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
        
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:


In [26]:
#Ayuda "eliminacion_bloques"
help(fQR.eliminacion_bloques)

Help on function eliminacion_bloques in module funciones_factorizacion_QR:

eliminacion_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que usa la eliminación por bloques para resolver un sistema de ecuaciones Ax=b.
    
    Parámetros:                            
      A     numpy.ndarray que representa la matriz (mxn) que representan loc coeficientes del sistema de      ecuaciones
      b     numpy.ndarray que representa el vector (mx1) que representan el lado derecho del sistema de       ecuaciones
      m1    Número de renglones del los bloques superiores del subconjunto a resolver
      n1    Número de columnas de los bloques del subconjunto a resolver
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
    
    Resultado:
      x   numpy.ndarray que representa la solución al sistema de ecuaciones Ax=b
    
    Este método consiste en eliminar un subconju

## Solución de Sistemas de Ecuaciones Lineales

### Ejemplos

### A) Sistema con soluciones únicas

In [27]:
#Generamos un sistema de ecuaciones lineales
m=12
n=12
A=np.round(fQR.crear_matriz_aleatoria(m,n,6,-6,False),2)
b = np.round(fQR.crear_matriz_aleatoria(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[ 4.56 -3.26 -1.35 -5.81 -5.71 -3.26  5.62  2.73  2.15 -0.52  4.33 -0.14]
 [ 4.14 -1.73  2.49  0.61 -4.13  3.64 -5.51 -1.09 -2.4  -5.93 -0.61 -4.25]
 [-5.23  3.74  3.17 -2.33 -4.75 -1.5   4.1  -2.1   5.32 -5.59 -2.8  -3.73]
 [ 2.24  0.99 -4.5   4.7   4.72 -4.91  0.68  2.77  4.64  4.99  3.12  4.77]
 [-1.01 -5.01 -1.59  4.37  2.9  -2.95  3.86 -1.91  4.12 -1.35  5.62  1.05]
 [ 3.32  5.48 -2.63  4.83 -1.84  3.    3.32 -2.2   6.    2.33  1.56 -3.21]
 [-4.97 -4.86  2.4   0.03  0.43  4.26  5.57 -4.93 -2.38 -4.11 -0.17 -1.93]
 [ 3.89  0.44  4.53  1.31  3.87  3.09  2.09  5.87 -5.01 -2.69  5.19 -2.32]
 [-5.37 -3.79  2.94 -5.67 -0.26  5.85  0.98 -1.86 -0.22 -4.79  0.64 -0.55]
 [-0.33  4.16 -4.33 -5.27  5.14  1.32  5.12  0.16 -4.33 -0.59 -3.47  2.17]
 [ 4.21 -1.89 -4.01  1.67 -0.65 -3.78 -0.77  3.97  1.46  1.11  1.19 -4.26]
 [ 1.4  -2.53  0.21 -5.81  1.53 -0.14 -5.8   3.24  1.87  5.48  1.57 -5.79]]
b: [[ 0.66]
 [-2.02]
 [ 5.04]
 [-3.51]
 [ 3.74]
 [ 4.64]
 [-5.53]
 [-4.37]
 [ 4.11]
 [-0.27]
 [-

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [28]:
np.linalg.solve(A,b)

array([[-3.45572795],
       [ 3.4290442 ],
       [-3.0314013 ],
       [-0.52511108],
       [-2.13104925],
       [-0.85729173],
       [-1.87630433],
       [-1.52914394],
       [-2.53808301],
       [-0.71875803],
       [ 5.82913249],
       [-0.37734315]])

Ahora usemos la factorización QR

In [30]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

SystemExit: La matriz A asociada al sistema de ecuaciones es singular

In [31]:
#Se prueba con un sistema más grande
m=10**3
n=10**3
A=np.round(fQR.crear_matriz_aleatoria(m,n,10,-10,False),2)
b=np.round(fQR.crear_matriz_aleatoria(m,1,10,-10,False),2)
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

SystemExit: La matriz A asociada al sistema de ecuaciones es singular

Por último usemos eliminación por bloques con QR

In [32]:
fQR.eliminacion_bloques(A,b)

C:\Users\GZARAZUA\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:2125: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


TypeError: Solucion_SEL_QR_nxn() takes 2 positional arguments but 3 were given

### B) Sistema sin solución

In [33]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [5]], dtype='d')
print("A:",A)
print("b:",b)

A: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
b: [[3.]
 [3.]
 [5.]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [34]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

* Marca un error: "Singular matrix"

Ahora usemos la eliminación por bloques usando la factorización QR

In [35]:
fQR.eliminacion_bloques(A,b)

SystemExit: A debe ser no singular

* Se concluye que es un sistema de rectas paralelas, por lo tanto no existe solúción.

### C) Sistema con infinitas soluciones

In [36]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [0]], dtype='d')
print("A:",A)
print("b:",b)

A: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
b: [[3.]
 [3.]
 [0.]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [37]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

* Marca un error: "Singular matrix"

Ahora usemos eliminación por bloques usando la factorización QR

In [38]:
fQR.eliminacion_bloques(A,b)

SystemExit: A debe ser no singular